### Functions for automating model selection through cross-validation.

In [2]:
import verde as vd
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import pyproj
from shapely import geometry

from sources import source_code as f

%matplotlib widget

# Tutorial

In [2]:
dic_cartas,dic_raw_metadata = f.get_region('100k','SC23_Z_A_IV','gama_1089')
data = dic_cartas['raw_data']['SC23_Z_A_IV']

importando dados geofisicos
# --- Iniciando seleção de área de estudo
1 folha cartográfica selecionada

Listando atributos dos dados geofisicos
# --- # Listagem de dados do aerolevantamento:  
Lista de atributos geofísicos = ['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO']
lista de atributos geograficos = ['LONGITUDE', 'LATITUDE']
lista de atributos projetados = ['X', 'Y']


/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)



# --- Início da iteração entre as folhas cartográficas #
# Folha de código: SC23_Z_A_IV
 Atualizando dados brutos em dic_cartas['raw_data']
 com 81859 pontos


In [8]:
print(f'Chaves do dicionario principal: {list(dic_cartas.keys())}')

Chaves do dicionario principal: ['region', 'region_proj', 'raw_data', 'interpolado', 'scores', 'lito_geof', 'mean_score']


In [10]:
print(f'Chaves do dicionario de metadados: {dic_raw_metadata.keys()}')

Chaves do dicionario de metadados: dict_keys(['Metadata', 'Lista_id', 'Lista_at_geof', 'Lista_at_geog', 'Lista_at_proj', 'Percentiles', 'Malha_cartografica'])


In [11]:
dic_cartas['raw_data']['SC23_Z_A_IV']

,X,Y,MDT,KPERC,eU,eth,CTCOR,THKRAZAO,UKRAZAO,UTHRAZAO,LONGITUDE,LATITUDE
2512,500020.25,8786155.43,477.68,0.35,1.25,16.12,1182.85,45.2580,3.5190,0.0778,-44.999815,-10.980692
2513,500103.83,8786158.77,476.83,0.63,1.24,14.00,1143.89,22.1205,1.9707,0.0891,-44.999050,-10.980662
2514,500187.24,8786161.64,476.58,1.02,1.38,12.02,1165.74,11.8674,1.3503,0.1138,-44.998286,-10.980636
2515,500270.50,8786163.59,470.91,1.41,1.51,11.31,1257.68,7.9624,1.0587,0.1330,-44.997524,-10.980618
2516,500353.54,8786165.69,464.63,1.71,1.65,11.14,1357.00,6.5584,0.9790,0.1493,-44.996764,-10.980599
...,...,...,...,...,...,...,...,...,...,...,...,...
264653,554619.00,8838949.65,471.69,0.95,1.47,15.02,1327.04,15.8524,1.5393,0.0971,-44.500850,-10.502821
264654,554618.94,8839028.43,463.98,0.80,1.17,12.29,1087.91,15.2830,1.4522,0.0950,-44.500852,-10.502108
264655,554618.52,8839107.80,454.86,0.63,1.10,9.72,881.47,15.7108,1.7910,0.1140,-44.500857,-10.501391
264656,554617.97,8839187.24,450.46,0.57,1.19,8.10,784.10,14.0517,2.0510,0.1460,-44.500863,-10.500672


In [12]:
data = dic_cartas['raw_data']['SC23_Z_A_IV']

In [13]:
coordinates = (data.X.values, data.Y.values)
coordinates

(array([500020.25, 500103.83, 500187.24, ..., 554618.52, 554617.97,
        554617.27]),
 array([8786155.43, 8786158.77, 8786161.64, ..., 8839107.8 , 8839187.24,
        8839266.99]))

In [14]:
data.describe(percentiles=(0.01,0.1,0.25,0.50,0.75,0.995)).T

,count,mean,std,min,1%,10%,25%,50%,75%,99.5%,max
X,81859.0,5.272457e+05,15881.461081,5.000053e+05,5.005883e+05,5.051369e+05,5.135953e+05,5.271237e+05,5.411101e+05,5.546207e+05,5.546952e+05
Y,81859.0,8.811616e+06,16019.792057,8.783976e+06,8.784576e+06,8.789346e+06,8.797549e+06,8.811636e+06,8.825717e+06,8.839012e+06,8.839305e+06
MDT,81859.0,5.198798e+02,79.852517,3.493300e+02,3.729400e+02,4.267980e+02,4.636400e+02,5.039200e+02,5.689350e+02,6.969500e+02,7.062200e+02
KPERC,81859.0,6.134073e-01,0.520534,-2.300000e-01,-1.000000e-02,1.100000e-01,2.200000e-01,4.600000e-01,8.700000e-01,2.470000e+00,3.870000e+00
eU,81859.0,1.544812e+00,0.756139,-1.030000e+00,2.200000e-01,7.100000e-01,1.030000e+00,1.450000e+00,1.950000e+00,4.460000e+00,1.214000e+01
eth,81859.0,1.406115e+01,6.582828,9.400000e-01,4.270000e+00,6.930000e+00,9.170000e+00,1.251000e+01,1.759000e+01,3.413420e+01,5.434000e+01
CTCOR,81859.0,1.181804e+03,456.726630,1.037700e+02,4.346674e+02,6.410660e+02,8.127350e+02,1.130380e+03,1.499520e+03,2.727711e+03,3.924330e+03
THKRAZAO,81859.0,6.933198e+01,145.286506,1.998000e+00,5.203834e+00,8.162980e+00,1.256500e+01,2.733780e+01,5.894520e+01,1.056563e+03,1.892694e+03
UKRAZAO,81859.0,7.116611e+00,13.503018,2.230000e-02,2.710320e-01,8.392000e-01,1.476000e+00,3.114800e+00,6.680850e+00,9.520675e+01,2.173391e+02
UTHRAZAO,81859.0,1.201141e-01,0.057837,2.300000e-03,2.525800e-02,6.080000e-02,8.090000e-02,1.093000e-01,1.482000e-01,3.399710e-01,9.555000e-01


In [15]:
print(f"Valores negativos no canal do K: {data.query('KPERC < 0')['KPERC'].count()}")
print(f"Valores negativos no canal do U: {data.query('eU < 0')['eU'].count()}")
print(f"Valores negativos no canal do Th: {data.query('eth < 0')['eth'].count()}")
print(f"Valores negativos no canal das CT: {data.query('CTCOR < 0')['CTCOR'].count()}")

Valores negativos no canal do K: 961
Valores negativos no canal do U: 177
Valores negativos no canal do Th: 0
Valores negativos no canal das CT: 0


In [16]:
data.loc[data.KPERC < 0, 'KPERC'] = 0
data.loc[data.eU < 0, 'eU'] = 0

In [17]:
print(data.query('KPERC < 0')['KPERC'].count())
print(data.query('eU < 0')['eU'].count())

0
0


In [18]:
data_descrito = data.describe(percentiles= [0.01,0.1,0.25,0.5,0.75,0.995])

In [19]:
data_descrito.T

,count,mean,std,min,1%,10%,25%,50%,75%,99.5%,max
X,81859.0,5.272457e+05,15881.461081,5.000053e+05,5.005883e+05,5.051369e+05,5.135953e+05,5.271237e+05,5.411101e+05,5.546207e+05,5.546952e+05
Y,81859.0,8.811616e+06,16019.792057,8.783976e+06,8.784576e+06,8.789346e+06,8.797549e+06,8.811636e+06,8.825717e+06,8.839012e+06,8.839305e+06
MDT,81859.0,5.198798e+02,79.852517,3.493300e+02,3.729400e+02,4.267980e+02,4.636400e+02,5.039200e+02,5.689350e+02,6.969500e+02,7.062200e+02
KPERC,81859.0,6.139342e-01,0.519875,0.000000e+00,0.000000e+00,1.100000e-01,2.200000e-01,4.600000e-01,8.700000e-01,2.470000e+00,3.870000e+00
eU,81859.0,1.545084e+00,0.755532,0.000000e+00,2.200000e-01,7.100000e-01,1.030000e+00,1.450000e+00,1.950000e+00,4.460000e+00,1.214000e+01
eth,81859.0,1.406115e+01,6.582828,9.400000e-01,4.270000e+00,6.930000e+00,9.170000e+00,1.251000e+01,1.759000e+01,3.413420e+01,5.434000e+01
CTCOR,81859.0,1.181804e+03,456.726630,1.037700e+02,4.346674e+02,6.410660e+02,8.127350e+02,1.130380e+03,1.499520e+03,2.727711e+03,3.924330e+03
THKRAZAO,81859.0,6.933198e+01,145.286506,1.998000e+00,5.203834e+00,8.162980e+00,1.256500e+01,2.733780e+01,5.894520e+01,1.056563e+03,1.892694e+03
UKRAZAO,81859.0,7.116611e+00,13.503018,2.230000e-02,2.710320e-01,8.392000e-01,1.476000e+00,3.114800e+00,6.680850e+00,9.520675e+01,2.173391e+02
UTHRAZAO,81859.0,1.201141e-01,0.057837,2.300000e-03,2.525800e-02,6.080000e-02,8.090000e-02,1.093000e-01,1.482000e-01,3.399710e-01,9.555000e-01


In [20]:
list_canal = dic_raw_metadata['Lista_at_geof']
print(f"Lista de atributos: {list_canal}")

Lista de atributos: ['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO']


In [21]:
# LISTA DE TITULOS 
titles =   ['MDT (m)',
            'K (%)',
            'U (ppm)',
            'Th (ppm)',
            'Contagem Total (μR/h)',
            'Th (ppm) / K (%)',
            'U (ppm) / K (%)',
            'U (ppm) / Th (ppm)']

# DICIONARIO DE TITULOS
dic_titles = {}
for f, t in zip(list_canal, titles):
    dic_titles[f] = t

dic_titles

{'MDT': 'MDT (m)',
 'KPERC': 'K (%)',
 'eU': 'U (ppm)',
 'eth': 'Th (ppm)',
 'CTCOR': 'Contagem Total (μR/h)',
 'THKRAZAO': 'Th (ppm) / K (%)',
 'UKRAZAO': 'U (ppm) / K (%)',
 'UTHRAZAO': 'U (ppm) / Th (ppm)'}

In [22]:
def raw_plot(data):
    fig, axs = plt.subplots(nrows = 2, ncols = 4, figsize = (9, 9))

    X,Y = data.X,data.Y

    for ax, f in zip(axs.flat, dic_titles):
        vmin=data_descrito[f]['1%']
        vmax=data_descrito[f]['99.5%']

        g = ax.scatter(c=data[f], x = X,
                                y = Y,
                                cmap = 'rainbow',
                                s    = 0.5,
                                vmax =vmax, vmin=vmin)

        fig.colorbar(g, ax = ax)
        ax.set_title(str(dic_titles[f]), size =9)
        ax.axis('scaled')

    #fig.delaxes(axs[2][1])

    #plt.tight_layout()

In [23]:
raw_plot(data)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
# Block Reduction
reducer_mean_490 = vd.BlockReduce(np.mean, spacing= 490)
reducer_mean_499 = vd.BlockReduce(np.mean, spacing= 499)
reducer_mean_500 = vd.BlockReduce(np.mean, spacing= 500)

In [25]:
# Block Reduction
reducer_median_490 = vd.BlockReduce(np.median, spacing= 490)
reducer_median_499 = vd.BlockReduce(np.median, spacing= 499)
reducer_median_500 = vd.BlockReduce(np.median, spacing= 500)

In [26]:
lista_blocks = [reducer_mean_490,reducer_mean_499,reducer_mean_500,
                reducer_median_490,reducer_median_499,reducer_median_500]

In [29]:
# Ploting the Reduced data
fig, axs = plt.subplots(nrows = 2, ncols = 3, figsize = (16, 16),sharex='all',sharey='all')

for ax, f in zip(axs.flat, lista_blocks):
    b_coords, b_eU = f.filter(coordinates, data.eU)

    g = ax.scatter(c    = b_eU, x = b_coords[0],
                                y = b_coords[1],
                             cmap = 'rainbow',
                             s    = 5)
    fig.colorbar(g,ax=ax)
    ax.set_title(f,size=4)
    ax.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
reducer_mean_490 = vd.BlockReduce(np.mean, spacing= 499)
b_coords, b_UTHRAZAO = reducer_mean_490.filter(coordinates, data.UTHRAZAO)

In [31]:
spline = vd.Spline()

In [32]:
spline.fit(b_coords, b_UTHRAZAO)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Spline()

In [33]:
predicted = spline.predict(coordinates)

In [58]:
plt.figure(figsize=(8,8))
plt.scatter(coordinates[0],coordinates[1],
            c=predicted,
            cmap='rainbow',
            s=1)
plt.axis('scaled')
plt.title('Valores preditos para os mesmos pontos amostrados')
plt.colorbar(label='UTHRAZAO')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
residuals = data.UTHRAZAO - predicted

In [60]:
scale = vd.maxabs(residuals)

plt.figure(figsize=(7,7))

plt.scatter(coordinates[0],coordinates[1],
            c=residuals,
            cmap='RdBu_r',
            s=2,
            vmin=-scale,vmax=scale)
plt.axis('scaled')
plt.title('Residuals: Dado amostrado - Dado predito')
plt.colorbar(label='UTHRAZAO')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
region = vd.get_region(coordinates)
grid_coords = vd.grid_coordinates(region, spacing = 250,pixel_register=True)

In [61]:
grid_UTHRAZAO = spline.predict(grid_coords)

In [62]:
plt.figure(figsize=(8,8))
plt.scatter(grid_coords[0], grid_coords[1],
            c=grid_UTHRAZAO,
            cmap='rainbow',
            s=2)
plt.axis('scaled')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
grid = spline.grid(spacing=250, data_names=['UTHRAZAO'])
grid

<xarray.Dataset>
Dimensions:   (northing: 221, easting: 219)
Coordinates:
  * easting   (easting) float64 5.001e+05 5.003e+05 ... 5.544e+05 5.547e+05
  * northing  (northing) float64 8.784e+06 8.784e+06 ... 8.839e+06 8.839e+06
Data variables:
    UTHRAZAO  (northing, easting) float64 0.1177 0.1284 0.1457 ... 0.0906 0.1254
Attributes:
    metadata:  Generated by Spline()

In [64]:
plt.figure()
grid.UTHRAZAO.plot(cmap='rainbow')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(499931.6324672345, 554778.1632470512, 8784061.033629868, 8839209.242560606)

In [67]:
grid = vd.distance_mask(coordinates, maxdist=500, grid=grid)
grid

grid.UTHRAZAO.plot(figsize=(7,7),cmap='rainbow')
plt.axis('scaled')

/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/xarray/plot/utils.py:429: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  _, ax = plt.subplots(figsize=figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(499931.6324672345, 554778.1632470512, 8784061.033629868, 8839209.242560606)

In [87]:
chain = vd.Chain([
    ('trend',  vd.Trend(degree=1)),
    ('reduce', vd.BlockReduce(np.mean, spacing=1000)),
    ('spline', vd.Spline())
])

In [88]:
chain.fit(coordinates, data.UTHRAZAO)

Chain(steps=[('trend', Trend(degree=1)),
             ('reduce',
              BlockReduce(reduction=<function mean at 0x7f8ff81cb3a0>,
                          spacing=1000)),
             ('spline', Spline())])

In [89]:
grid = chain.grid(spacing=250, data_names=['UTHRAZAO'])
grid_mask = vd.distance_mask(coordinates, maxdist=500, grid=grid)
grid_mask

<xarray.Dataset>
Dimensions:   (northing: 222, easting: 220)
Coordinates:
  * easting   (easting) float64 5e+05 5.003e+05 ... 5.544e+05 5.547e+05
  * northing  (northing) float64 8.784e+06 8.784e+06 ... 8.839e+06 8.839e+06
Data variables:
    UTHRAZAO  (northing, easting) float64 0.1301 0.1312 0.133 ... 0.1108 0.1121
Attributes:
    metadata:  Generated by Chain(steps=[('trend', Trend(degree=1)),\n       ...

In [114]:
df_THKRAZAO=grid_mask.to_dataframe()
df_THKRAZAO_descrito = df_THKRAZAO.describe(percentiles=(0.01,0.1,0.25,0.5,0.75,0.995))

In [115]:
df_THKRAZAO_descrito

,UTHRAZAO
count,48840.000000
mean,0.120143
std,0.038541
min,0.029307
1%,0.064182
10%,0.077894
25%,0.091941
50%,0.115054
75%,0.139801
99.5%,0.267650


In [91]:
grid_mask.UTHRAZAO.plot(figsize=(8,8), cmap='rainbow',vmin=df_THKRAZAO_descrito.UTHRAZAO.min(),
                                                      vmax=df_THKRAZAO_descrito.UTHRAZAO.max())
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(499880.4371004566, 554820.1128995434, 8783850.761764705, 8839429.898235295)

### Model Validation

### Escondendo 10% dos dados antes de inserir no chain.fit()

In [95]:
train, test = vd.train_test_split(coordinates, data.UTHRAZAO,
                                 test_size = 0.1)

plt.figure(figsize=(12,12))
plt.title(label='Dados Escondidos(Vermelho) Dados inseridos no modelo de Splines(Azul)')
plt.plot(train[0][0], train[0][1], '.b', markersize=4)
plt.plot(test[0][0], test[0][1], '.r', markersize=4)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(497270.8025, 557429.7475, 8781209.501, 8842071.159)

In [73]:
chain.fit(*train)
# *train   ---> chain.fit(train[0], train[1])

chain.score(*test)


0.6673176584341012

In [118]:
chain = vd.Chain([
    ('trend',  vd.Trend(degree=4)),
    ('reduce', vd.BlockReduce(np.mean, spacing=1500)),
    ('spline', vd.Spline())
])

In [119]:
train, test = vd.train_test_split(coordinates, data.UTHRAZAO, test_size=0.1, spacing=500)

plt.figure(figsize=(9,9))
plt.plot(train[0][0], train[0][1], '.b', markersize=10)
plt.plot(test[0][0], test[0][1], '.r', markersize=10)
plt.axis('scaled')

chain.fit(*train)

chain.score(*test)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.339774914835025

# Script

In [1]:
from sources import source_code as f

In [8]:
dic_cartas['region_proj'][i]

(500000.0, 527356.0359910944, 8811651.835175423, 8839304.90587394)

In [4]:
dic_cartas,dic_raw_metadata = f.get_region('100k','SC23_Z_A_IV','gama_1089')

lista_id = dic_raw_metadata['Lista_id']
print(list(lista_id))

for i in lista_id:
    print(f"folha {i}")
    data = dic_cartas['raw_data'][i]

    coordinates = (data.X.values, data.Y.values)
    region = dic_cartas['region_proj'][i]

    chain = vd.Chain([('trend',  vd.Trend(degree=1)),
                     ('reduce', vd.BlockReduce(np.mean, spacing=1000)),
                     ('spline', vd.Spline())
                    ])

    cv     = vd.BlockKFold(spacing=500,
                            n_splits=10,
                            shuffle=True)

    scores = vd.cross_val_score(chain,
                            coordinates,
                            data.eU,
                            cv=cv,
                            delayed=True)

    import dask
    mean_score = dask.delayed(np.mean)(scores)
    print("Delayed mean:", mean_score)

    mean_score = mean_score.compute()

    print(mean_score)

    print("")

importando dados geofisicos
# --- Iniciando seleção de área de estudo
1 folha cartográfica selecionada

Listando atributos dos dados geofisicos
# --- # Listagem de dados do aerolevantamento:  
Lista de atributos geofísicos = ['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO']
lista de atributos geograficos = ['LONGITUDE', 'LATITUDE']
lista de atributos projetados = ['X', 'Y']


/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)



# --- Início da iteração entre as folhas cartográficas #
# Folha de código: SC23_Z_A_IV
 Atualizando dados brutos em dic_cartas['raw_data']
 com 81859 pontos
['SC23_Z_A_IV']
folha SC23_Z_A_IV
Delayed mean: Delayed('mean-cdb89650-64c9-4f20-aa81-11a8e104faff')
0.511437021127595



In [18]:
dic_cartas,dic_raw_metadata = f.get_region('100k','SC23_Z_A_IV','gama_1089')

lista_id = dic_raw_metadata['Lista_id']
print(list(lista_id))

for i in lista_id:
    print(f"folha {i}")
    data = dic_cartas['raw_data'][i]

    coordinates = (data.X.values, data.Y.values)
    region = dic_cartas['region_proj'][i]

    chain = vd.Chain([('trend',  vd.Trend(degree=3)),
                     ('reduce', vd.BlockReduce(np.mean, spacing=2000)),
                     ('spline', vd.Spline())
                    ])

    cv     = vd.BlockKFold(spacing=100,
                            n_splits=50,
                            shuffle=True)

    scores = vd.cross_val_score(chain,
                            coordinates,
                            data.eU,
                            cv=cv,
                            delayed=True)

    import dask
    mean_score = dask.delayed(np.mean)(scores)
    print("Delayed mean:", mean_score)

    mean_score = mean_score.compute()

    print(mean_score)

    print("")

importando dados geofisicos
# --- Iniciando seleção de área de estudo
1 folha cartográfica selecionada

Listando atributos dos dados geofisicos
# --- # Listagem de dados do aerolevantamento:  
Lista de atributos geofísicos = ['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO']
lista de atributos geograficos = ['LONGITUDE', 'LATITUDE']
lista de atributos projetados = ['X', 'Y']


/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)



# --- Início da iteração entre as folhas cartográficas #
# Folha de código: SC23_Z_A_IV
 Atualizando dados brutos em dic_cartas['raw_data']
 com 81859 pontos
['SC23_Z_A_IV']
folha SC23_Z_A_IV
Delayed mean: Delayed('mean-d5ad70cd-a5ef-46f1-83e5-bc018883158a')
0.4794616449012649



In [7]:
dic_cartas.keys()

dict_keys(['region', 'region_proj', 'raw_data', 'interpolado', 'scores', 'lito_geof', 'mean_score'])

In [10]:
dic_cartas['interpolado']['SC23_Z_A_IV']

''

In [6]:
lista_id = dic_raw_metadata['Lista_id']
print(list(lista_id))
lista_geof = dic_raw_metadata['Lista_at_geof']
print(list(lista_geof))

['SC23_Z_A_IV']
['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO']


In [14]:
for index in lista_id:
    print(f"folha {index}")
    data = dic_cartas['raw_data'][index]

    coordinates = (data.X.values, data.Y.values)
    region = dic_cartas['region_proj'][index]

    chain = vd.Chain([('trend',  vd.Trend(degree=1)),
                     ('reduce', vd.BlockReduce(np.mean, spacing=2000)),
                     ('spline', vd.Spline())
                    ])


    print(f"# Folha de código: {index}")
    print(f" Atualizando dados brutos em dic_cartas['raw_data']")
    x = {index:data}
    data_list[0]['raw_data'].update(x) 
    print(f" com {len(data)} pontos de contagens radiométricas coletados com linhas de voo de {spacing} metros")
    
    # ADICIONANDO ATRIBUTOS GEOFÍSICOS AO DICIONÁRIO INTERPOLADO
    interpolado={}
    for atributo in lista_geof:
        x = {atributo:''}
        interpolado.update(x)
    #print(f" Construindo dic_cartas['interpolado'] vazio com os atributos geofísicos")
        
    # ADICIONANDO ATRIBUTOS AO DICIONÁRIO SCORES
    scores={}
    for atributo in lista_geof:
        y = {atributo:''}
        scores.update(y)

    mean_score={}
    for atributo in lista_geof:
        x = {atributo:''}
        mean_score.update(x)    

    #print(f" Construindo dicionário vazio de score do cross validation")
    
    for i in lista_geof:
        cv     = vd.BlockKFold(spacing=500,
                                n_splits=10,
                                shuffle=True)

        scores[i] = vd.cross_val_score(chain,
                                coordinates,
                                data[i],
                                cv=cv,
                                delayed=True)

        import dask
        
        mean_score[i] = dask.delayed(np.mean)(scores[i])
        print("Delayed mean:", mean_score)

        mean_score[i] = mean_score[i].compute()
        print(mean_score)


        y = {index:scores}
        dic_cartas['scores'].update(y)
        x = {index:mean_score}
        dic_cartas['mean_score'].update(x)
        print(f"# Folha {index} atualizada ao dicionário")

        print("")

folha SC23_Z_A_IV
# Folha de código: SC23_Z_A_IV
 Atualizando dados brutos em dic_cartas['raw_data']


NameError: name 'data_list' is not defined

In [ ]:
grid_coords = vd.grid_coordinates(region, spacing = 250,pixel_register=True)
chain.fit(grid_coords)